In [3]:
import numpy as np
np.random.seed(1337)  # for reproducibility
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Dense
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.layers import SimpleRNN, Activation, Dense

Using TensorFlow backend.


In [5]:
SHIFT=5
TIME_STEPS = 40    
INPUT_SIZE = 12    
BATCH_SIZE = 28
OUTPUT_SIZE = 1
BATCH_START = 0
CELL_SIZE = 50
LR = 0.001
TRAIN_TEST_SEP = 80*40+1
my_data = np.genfromtxt("cleandata2.csv",
                        delimiter=',', dtype='float32')

In [46]:
my_data.shape

(4321, 14)

In [6]:
X_train = my_data[1:TRAIN_TEST_SEP,1:13]
X_test= my_data[TRAIN_TEST_SEP:,1:13]
X_test = my_data[TRAIN_TEST_SEP:,1:13].reshape(
    (-1, TIME_STEPS, INPUT_SIZE))
y_train = my_data[1:TRAIN_TEST_SEP,13]
y_test = my_data[TRAIN_TEST_SEP:,13].reshape(
    (-1, TIME_STEPS, 1))
print(y_train)

[ 0.  0.  0. ...,  0.  0.  1.]


In [56]:
X_train

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0., ...,  0.,  1.,  1.],
       [ 0.,  0.,  0., ...,  0.,  1.,  1.],
       [ 1.,  0.,  0., ...,  0.,  1.,  1.]], dtype=float32)

In [57]:
X_train.shape

(3200, 12)

In [63]:
y_test = my_data[TRAIN_TEST_SEP:,13]
y_test.reshape

array([ 0.,  1.,  0., ...,  0.,  0.,  0.], dtype=float32)

In [7]:
def get_batch():
    global BATCH_START, TIME_STEPS,SHIFT
    # xs shape (50batch, 20steps)
    X_batch = X_train[BATCH_START:BATCH_START + TIME_STEPS *
                      BATCH_SIZE,:].reshape((BATCH_SIZE, TIME_STEPS, INPUT_SIZE))
    Y_batch = y_train[BATCH_START:BATCH_START + TIME_STEPS *
                      BATCH_SIZE, ].reshape((BATCH_SIZE, TIME_STEPS, 1))
    if (BATCH_START + 2*TIME_STEPS * BATCH_SIZE) >= (X_train.shape[0]):
        BATCH_START=SHIFT
        SHIFT+=5
        # BATCH_START = 0
    else:
        BATCH_START += TIME_STEPS *BATCH_SIZE
    return [X_batch, Y_batch]

In [8]:
model = Sequential()
model.add(LSTM(
    batch_input_shape=(BATCH_SIZE,TIME_STEPS,INPUT_SIZE),
    output_dim=CELL_SIZE,
    return_sequences=True,
    stateful=True
))

model.add(TimeDistributed(Dense(OUTPUT_SIZE)))
model.add(Activation('sigmoid'))
adam=Adam(LR)
model.compile(optimizer=adam, loss='mse',
              metrics=['accuracy'])
print('Training ------------')

/Users/amandachen/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, stateful=True, batch_input_shape=(28, 40, 1..., units=50)`
  


Training ------------


In [9]:
for step in range(100):
    X_batch, Y_batch= get_batch()
    cost = model.train_on_batch(X_batch, Y_batch)
    
   
    if step % 10 == 0:
        #X_test, y_test=get_test()
        # X_test = X_test.astype('float32')
        # y_test = y_test.astype('int')
        #print(y_test.shape[0].dtype)
        pred = model.predict(X_test, BATCH_SIZE)
        print(pred)
        cost ,accuracy= model.evaluate(X_test, y_test,batch_size=28, verbose=False)
        #error because we need a prediction each time step
        print('test cost: ', cost, 'accuracy:', accuracy)

[[[ 0.4778268 ]
  [ 0.47224018]
  [ 0.46185613]
  ..., 
  [ 0.45362619]
  [ 0.44709694]
  [ 0.44105014]]

 [[ 0.4814176 ]
  [ 0.4753693 ]
  [ 0.47430611]
  ..., 
  [ 0.49972752]
  [ 0.49051288]
  [ 0.48825264]]

 [[ 0.51318008]
  [ 0.50710613]
  [ 0.47287282]
  ..., 
  [ 0.50104535]
  [ 0.50456727]
  [ 0.49127701]]

 ..., 
 [[ 0.50507367]
  [ 0.49318516]
  [ 0.49248168]
  ..., 
  [ 0.52263373]
  [ 0.51009852]
  [ 0.49956742]]

 [[ 0.51026976]
  [ 0.50612605]
  [ 0.49949756]
  ..., 
  [ 0.50175118]
  [ 0.50744927]
  [ 0.50191355]]

 [[ 0.4955928 ]
  [ 0.50202465]
  [ 0.50606018]
  ..., 
  [ 0.50517839]
  [ 0.49538043]
  [ 0.4980526 ]]]
test cost:  0.531079590321 accuracy: 0.591071426868
[[[ 0.43653461]
  [ 0.41821724]
  [ 0.39784032]
  ..., 
  [ 0.3610476 ]
  [ 0.35821763]
  [ 0.35546401]]

 [[ 0.40224418]
  [ 0.39681375]
  [ 0.3956854 ]
  ..., 
  [ 0.41968909]
  [ 0.40734193]
  [ 0.40608594]]

 [[ 0.42014852]
  [ 0.42157444]
  [ 0.39278409]
  ..., 
  [ 0.43759391]
  [ 0.44260931]
  [ 0

In [11]:
pred=model.predict(X_test,BATCH_SIZE)

In [13]:
import matplotlib.pyplot as plt
plt.plot(X_test,pred)

ValueError: x and y can be no greater than 2-D, but have shapes (28, 40, 12) and (28, 40, 1)